In [ ]:
# https://stackoverflow.com/a/67029719/7782
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()

@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic('bash', '', cell.format(**globals()))

In [ ]:
import ezid_client_tools as ect
from ezid_client_tools.utils import ANVL

import argparse
import settings

import os
import types


In [2]:
import structured_ezid as sezid

In [ ]:
EZID_USER = os.environ.get('EZID_USER')
EZID_PASSWD = os.environ.get('EZID_PASSWD')

if (EZID_USER is None) or (EZID_PASSWD is None):
    import settings
    EZID_USER = settings.EZID_USER
    EZID_PASSWD = settings.EZID_PASSWD


In [ ]:
cclient = ect.ConsoleClient()

cclient.args.credentials = f"{EZID_USER}:{EZID_PASSWD}"
cclient.args.server = "p"

cclient.args.operation = ['view', 'ark:/28722/k2154wc6r']

cclient.operation()

In [ ]:
client = ect.Client()

client.args.credentials = f"{EZID_USER}:{EZID_PASSWD}"
client.args.server = "p"

client.args.operation = ['view', 'ark:/28722/k2154wc6r']

In [ ]:
r = client.operation()
ANVL.parse_anvl_str(r.encode('utf-8'))

In [ ]:
%%pybash

ezid p "{EZID_USER}:{EZID_PASSWD}" view ark:/28722/k2154wc6r 

In [ ]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" -d format=anvl -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-01-01T00:00:00Z \
  https://ezid.cdlib.org/download_request

In [ ]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" \
  -d format=csv \
  -d column=_id \
  -d column=erc.who \
  -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-01-01T00:00:00Z \
  https://ezid.cdlib.org/download_request


In [ ]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" \
  -d format=csv \
  -d column=_id \
  -d column=erc.who \
  -d column=erc.what \
  -d column=erc.when \
  -d column=_owner \
  -d column=_ownergroup \
  -d column=_created \
  -d column=_updated \
  -d column=_profile \
  -d column=_target \
  -d column=_status \
  -d column=_export \
  -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-04-01T00:00:00Z \
  https://ezid.cdlib.org/download_request



In [ ]:
import requests
url = 'https://ezid.cdlib.org/download_request'
data = {
    'format': 'csv',
    'column': ['_id', 'erc.who', 'erc.what', 'erc.when', '_owner', '_ownergroup', '_created', '_updated', '_profile', '_target', '_status', '_export'],
    'type': 'ark',
    'permanence': 'real',
    'createdAfter': '2023-04-01T00:00:00Z'
}

response = requests.post(url, auth=(EZID_USER, EZID_PASSWD), data=data)

if response.status_code == 200:
    print(response.text)
else:
    print(f"Error: {response.status_code}")

In [ ]:
response.status_code, response.text

# Test arks

In [ ]:
TEST_NAAN = "99999"
TEST_SHOULDER = "fk4"


In [ ]:
# write a public fake ARK

seems like we write arbitrary metadata fields. To delete the key, set it to ""


In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" create! ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5 \
    _status public \
    erc.who "Raymond Yee" \
    erc.what "testing ark 00/05" \
    erc.when "(:unkn) Unknown time" \
    ry.who "" \
    profile erc

In [ ]:
from itertools import chain

client.args.server = "s"

metadata_ = {
  "ry.who": "Raymond Yee",
  "erc.who": "Nobody!",
  "profile": "erc",
  "erc.what": "testing ark 00/05",
  "erc.when": "(:unkn) Unknown time"
}


client.args.operation = ['create!', f'ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5'] + list(chain(*metadata_.items()))
r = client.operation()
r

In [ ]:
# testing prefix matching



In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" view ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5


In [ ]:
%%pybash

ezid -h

# Reserved IDs: creating and deleting

In [ ]:
%%pybash

# test shoulder and reserved id
# _status = reserved

ezid s "{EZID_USER}:{EZID_PASSWD}" create! "ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a7" \
    _status reserved \
    erc.who "Anthony Tuck" \
    erc.what "Photo 19660118 from Italy/Poggio Civitate/Tesoro/Tesoro 2F/1966, ID:476/PC 19660118" \
    erc.when "2017-10-04" \
    profile erc

In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" view "ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a7"

In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" delete "ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a6"

# Preregistration shoulder

In [ ]:
# New shoulder

OC_NAAN = "28722"
OC_PREREG_SHOULDER = "r2"


In [ ]:
# issue of allowable characters

from urllib.parse import quote, unquote
import re


quote("A.B")

test_string = '12hello.there'
re.match(r'^[\w\-_.~]*$', test_string)

def uri_unreserved(s):
    """
    
    
    """
    return re.match(r'^[\w\-_.~]*$', s) is not None



# lowercase, numbers



quote("我") == '%E6%88%91'

uri_unreserved(test_string)


In [ ]:
# ark string checker
# check on elimination of hyphens and terminal periods and /

"""

"""

In [ ]:
class EZIDIdentifier():
    pass

class DOIIdentifier():
    pass

class ARKIdentifier(EZIDIdentifier):
    def __init__(self, naan, shoulder, postfix, new_style=False):
        self.naan = naan
        self.shoulder = shoulder
        self.postfix = postfix
        self.new_style = new_style
    def __repr__(self):
        label = "ark:" if self.new_style else "ark:/"
        return f"{label}{self.naan}/{self.shoulder}{self.postfix}"
            

class Client2(ect.Client):
    def create_identifier(self, id_, metadata=None, update=True):
        self.args.operation = [f"""create{"!" if update else ""}""", id_] 
        if metadata is not None:
            self.args.operation += list(chain(*metadata_.items()))
        return self.operation()

    def view_identifier(self, id_, prefix_matching=False):
        self.args.operation = [f"""view{"!" if prefix_matching else ""}""", id_]
        r = client.operation()
        return ANVL.parse_anvl_str(r.encode('utf-8'))
    

In [ ]:
from typing import Union, Optional


class EZIDIdentifier:
    pass


class DOIIdentifier:
    pass


class ARKIdentifier(EZIDIdentifier):
    def __init__(self, naan: str, shoulder: str, postfix: str, new_style: bool = False):
        self.naan = naan
        self.shoulder = shoulder
        self.postfix = postfix
        self.new_style = new_style

    def __repr__(self):
        label = "ark:" if self.new_style else "ark:/"
        return f"{label}{self.naan}/{self.shoulder}{self.postfix}"


class Client2(ect.Client):
    def create_identifier(self, id_: Union[str, ARKIdentifier], metadata: Optional[dict] = None, update: bool = True):
        self.args.operation = [f"""create{"!" if update else ""}""", id_]
        if metadata is not None:
            self.args.operation += list(chain(*metadata.items()))
        return self.operation()

    def view_identifier(self, id_: Union[str, ARKIdentifier], prefix_matching: bool = False):
        self.args.operation = [f"""view{"!" if prefix_matching else ""}""", id_]
        r = self.operation()
        return ANVL.parse_anvl_str(r.encode('utf-8'))

In [ ]:
client2 = Client2()
client2.args.credentials = f"{EZID_USER}:{EZID_PASSWD}"
client2.args.server = "s"

ark_ = ARKIdentifier(TEST_NAAN, TEST_SHOULDER, "/00/a5")
ark_

client2.view_identifier(str(ark_))

# Analyzing OpenContext arks

In [ ]:
import pandas as pd
from pandas import DataFrame, Series, Index
import numpy as np

In [ ]:
df = pd.read_csv("data/5lrRHNLMtGqBVN8d.csv")
df.head(2)

In [ ]:
len(df)

In [ ]:
df['_status'].value_counts()